In [1]:
import os

In [2]:
%pwd

'd:\\MAIN_PROJECTS\\DataScience_Project\\research'

In [3]:
os.chdir("../")
%pwd

'd:\\MAIN_PROJECTS\\DataScience_Project'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionconfig:
    root_dir: Path
    source_URL: str
    local_data_file: str
    unzip_dir: Path

In [5]:
from src.DataScience.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH,SCHEMA_FILE_PATH
from src.DataScience.utils.common import read_yaml,create_directories

In [6]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH
                 ):
        self.config= read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self)->DataIngestionconfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionconfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [7]:
import urllib.request as request
from src.DataScience import logger
import zipfile

class DataIngestionComponent:
    def __init__(self,config:DataIngestionconfig):
        self.config=config
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url = self.config.source_URL,
                filename= self.config.local_data_file,
            )
            logger.info(f"{filename} downloaded with following info: \n{headers}")
        else:
            logger.info(f"File already exists")
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        zip_path = self.config.local_data_file

        # 🔍 Validate before extracting
        if not zipfile.is_zipfile(zip_path):
            raise Exception(f"The file at '{zip_path}' is not a valid zip file. Check download or URL.")

        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

        logger.info(f"Successfully extracted ZIP to {unzip_path}")
            

In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion = DataIngestionComponent(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-06-21 17:48:54,787: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-21 17:48:54,789: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-21 17:48:54,792: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-06-21 17:48:54,794: INFO: common: Created directory at: artifacts]
[2025-06-21 17:48:54,795: INFO: common: Created directory at: artifacts/data_ingestion]
[2025-06-21 17:48:55,594: INFO: 2184058100: artifacts/data_ingestion/data.zip downloaded with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: BD8D:3E5C52:E941A:2D911F:6856A330
Accept-Ranges: bytes
Date: Sat, 21